## 1. UAV Data_Helpful

In [ ]:
import pandas as pd
import numpy as np
res = pd.read_csv("UAV_Helpful_5CV(SLS_opt#1_new).csv") 
res = res.iloc[:,:]
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
res

In [ ]:
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
df_acc

In [ ]:
col_name = res.columns[1:]
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
import bayesiantests as bt
import matplotlib.pyplot as plt

def stacked_bar(results, category_names):
    """
    Parameters
    ----------
    results : dict
        A mapping from question labels to a list of answers per category.
        It is assumed all lists contain the same number of entries and that
        it matches the length of *category_names*.
    category_names : list of str
        The category labels.
    """
    labels = list(results.keys())
    data = np.array(list(results.values()))
    data_cum = data.cumsum(axis=1)
    category_colors = plt.colormaps['RdYlGn'](
        np.linspace(0.15, 0.85, data.shape[1]))

    fig, ax = plt.subplots(figsize=(9.2, 5))
    ax.invert_yaxis()
    ax.xaxis.set_visible(False)
    ax.set_xlim(0, np.sum(data, axis=1).max())

    for i, (colname, color) in enumerate(zip(category_names, category_colors)):
        widths = data[:, i]
        starts = data_cum[:, i] - widths
        rects = ax.barh(labels, widths, left=starts, height=0.5,
                        label=colname, color=color)

        r, g, b, _ = color
        text_color = 'white' if r * g * b < 0.5 else 'darkgrey'
        ax.bar_label(rects, label_type='center', color=text_color)
    ax.legend(ncols=len(category_names), bbox_to_anchor=(0, 1),
              loc='upper left', fontsize='small')

    return fig, ax

In [ ]:
r = 0.001

In [ ]:
# Bayesian Sign Test

rope = r
baselines = ['Focal','Hard']
ours = ['SLS(0.01)']
dfs = [df_acc]

comp = []
basewin = []
draw = []
ourswin = []
z = 0
for i in range(len(ours)):
    for j in range(len(baselines)):
#         print(z)
        names = (baselines[j],ours[i])
#         print(names)
        comp.append(names)
        X = np.array(dfs[i][[baselines[j],ours[i]]])
        left, within, right = bt.signtest(X, rope=rope, verbose=True, names=names)
#         print(left, within, right)
        basewin.append(left)
        draw.append(within)
        ourswin.append(right)        
results = pd.DataFrame(comp, columns = ["Baseline","Ours"])
results["Basewin_prob"] = basewin
results["Draw_prob"] = draw
results["Ourswin_prob"] = ourswin
results